In [1]:
import pandas as pd
import yfinance as yf
import requests
import csv
import psycopg2
import sqlalchemy

print(pd.__version__)
print(yf.__version__)
print(requests.__version__)
print(csv.__version__)
print(psycopg2.__version__)
print(sqlalchemy.__version__)

2.3.3
0.2.66
2.32.5
1.0
2.9.11 (dt dec pq3 ext lo64)
2.0.44


In [1]:
# Read stock symbols from the file
with open('sp500_symbols.txt', 'r') as file:
    stock_symbols = [line.strip() for line in file.readlines()]
print(f"Loaded {len(stock_symbols)} stock symbols.")

Loaded 503 stock symbols.


In [4]:
# --------------------------------------------------------------
#  sp500_renew_data.py – Daily Update Script
# --------------------------------------------------------------
import pandas as pd
import yfinance as yf
import pickle, os, time
from datetime import datetime, timedelta

# ---------- SETTINGS ----------
CSV_FILE    = 'sp500_historical_data.csv'
SYMBOL_FILE = 'sp500_symbols.txt'
CHECKPOINT  = 'sp500_renew_progress.pkl'
END         = datetime.now()
START       = END - timedelta(days=731)  # Last two years days (covers weekends)
FAILED_TXT  = 'renew_failed.txt'

# ---------- 1. Load symbols ----------
with open(SYMBOL_FILE) as f:
    ALL_SYMBOLS = [s.strip() for s in f if s.strip()]

# ---------- 2. Load existing CSV ----------
existing_df = pd.read_csv(CSV_FILE, header=[0,1], index_col=0, parse_dates=True)
saved_symbols = set(existing_df.columns.levels[0])
print(f"Existing CSV: {len(saved_symbols)} symbols")

# ---------- 3. Resume checkpoint ----------
if os.path.exists(CHECKPOINT):
    with open(CHECKPOINT, 'rb') as f:
        prog = pickle.load(f)
    tried = set(prog['tried'])
    failed = prog['failed']
else:
    tried, failed = set(), []

to_fetch = [s for s in ALL_SYMBOLS if s not in tried]
print(f"Renewing {len(to_fetch)} symbols...")

# ---------- 4. Fetch function ----------
def fetch(sym: str, retries: int = 3, delay: float = 2.0):
    """Return a DataFrame or None.  `sym` is expected in Yahoo format ('.' → '-')"""
    sym_y = sym.replace('.', '-')               # <─ crucial for BRK.B, BF.B …
    for attempt in range(retries):
        try:
            df = yf.Ticker(sym_y).history(start=START, end=END, raise_errors=True)
            if df.empty:
                return None
            return df[['Open','High','Low','Close','Volume']]
        except Exception as e:
            if attempt < retries-1:
                time.sleep(delay)
            else:
                print(f"  {sym} → {sym_y} error: {e}")
    return None

# ---------- 5. Loop ----------
new_data = {}
for i, sym in enumerate(to_fetch, 1):
    print(f"[{i}/{len(to_fetch)}] {sym}", end=' ')
    df = fetch(sym)
    if df is not None:
        new_data[sym] = df
        print("OK")
    else:
        failed.append(sym)
        print("FAILED")
    
    tried.add(sym)
    with open(CHECKPOINT, 'wb') as f:  # Save after EVERY symbol
        pickle.dump({'tried': list(tried), 'failed': failed}, f)
    time.sleep(1)  # Gentle delay

# ---------- 6. Merge & Save ----------
if new_data:
    new_df = pd.concat(new_data, axis=1, keys=new_data.keys())
    # Align dates and merge (only new rows)
    combined = pd.concat([existing_df, new_df]).groupby(level=0).last()
    combined.to_csv(CSV_FILE)
    print(f"Renewed! CSV now up to {END.date()}")

# ---------- 7. RE-FETCH FAILED SYMBOLS (at the very end) ----------
if failed:
    print(f"\n=== Re-trying {len(failed)} failed symbols ===")
    still_failed = []
    refetch_data = {}

    for i, sym in enumerate(failed, 1):
        print(f"[{i}/{len(failed)}] {sym}", end=' ')
        df = fetch(sym, retries=5, delay=3)      # more patience on the second pass
        if df is not None:
            refetch_data[sym] = df
            print("RECOVERED")
        else:
            still_failed.append(sym)
            print("STILL FAILED")
        time.sleep(1.5)

    # merge any recovered symbols
    if refetch_data:
        rec_df = pd.concat(refetch_data, axis=1, keys=refetch_data.keys())
        combined = pd.concat([pd.read_csv(CSV_FILE, header=[0,1], index_col=0, parse_dates=True),
                              rec_df]).groupby(level=0).last()
        combined.to_csv(CSV_FILE)
        print(f"Re-fetch success – {len(refetch_data)} symbols added to CSV")

    # update failed list
    if still_failed:
        with open(FAILED_TXT, 'w') as f:
            f.write('\n'.join(still_failed))
        print(f"{len(still_failed)} symbols remain failed → {FAILED_TXT}")
    else:
        if os.path.exists(FAILED_TXT):
            os.remove(FAILED_TXT)
        print("All symbols recovered – failed list removed.")
else:
    print("\nNo failures in the main run.")

# ---------- 8. Clean up ----------
if os.path.exists(CHECKPOINT):
    os.remove(CHECKPOINT)

print("\n=== DONE ===")

Existing CSV: 501 symbols
Renewing 503 symbols...
[1/503] MMM OK
[2/503] AOS OK
[3/503] ABT OK
[4/503] ABBV OK
[5/503] ACN OK
[6/503] ADBE OK
[7/503] AMD OK
[8/503] AES OK
[9/503] AFL OK
[10/503] A OK
[11/503] APD OK
[12/503] ABNB OK
[13/503] AKAM OK
[14/503] ALB OK
[15/503] ARE OK
[16/503] ALGN OK
[17/503] ALLE OK
[18/503] LNT OK
[19/503] ALL OK
[20/503] GOOGL OK
[21/503] GOOG OK
[22/503] MO OK
[23/503] AMZN OK
[24/503] AMCR OK
[25/503] AEE OK
[26/503] AEP OK
[27/503] AXP OK
[28/503] AIG OK
[29/503] AMT OK
[30/503] AWK OK
[31/503] AMP OK
[32/503] AME OK
[33/503] AMGN OK
[34/503] APH OK
[35/503] ADI OK
[36/503] AON OK
[37/503] APA OK
[38/503] APO OK
[39/503] AAPL OK
[40/503] AMAT OK
[41/503] APTV OK
[42/503] ACGL OK
[43/503] ADM OK
[44/503] ANET OK
[45/503] AJG OK
[46/503] AIZ OK
[47/503] T OK
[48/503] ATO OK
[49/503] ADSK OK
[50/503] ADP OK
[51/503] AZO OK
[52/503] AVB OK
[53/503] AVY OK
[54/503] AXON OK
[55/503] BKR OK
[56/503] BALL OK
[57/503] BAC OK
[58/503] BAX OK
[59/503] BDX OK


In [3]:
# sp500_upload_simple.py
import getpass
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

# ====================== CONFIG ======================
CSV_PATH   = "sp500_historical_data.csv"
DB_USER    = "postgres"
DB_PASSWORD = getpass.getpass("Password: ")
DB_HOST    = "localhost"
DB_PORT    = "5432"
DB_NAME    = "bootcamp_2508_final_project"
TABLE_NAME = "sp500_historical_data"

# ====================== 1. READ & MELT ======================
print("Loading and melting CSV...")
df_wide = pd.read_csv(CSV_PATH, header=[0, 1], index_col=0, parse_dates=True, low_memory=False)
df_wide.columns = ['_'.join(col).strip() for col in df_wide.columns.values]
df_wide = df_wide.reset_index()

df_long = pd.melt(
    df_wide,
    id_vars='Date',
    value_vars=[c for c in df_wide.columns if c != 'Date'],
    var_name='symbol_metric',
    value_name='value'
)

df_long[['symbol', 'metric']] = df_long['symbol_metric'].str.split('_', expand=True)
df_long = df_long.drop(columns='symbol_metric').rename(columns={'Date': 'date'})

metric_map = {'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}
df_long['metric'] = df_long['metric'].map(metric_map)

df_long = df_long[['symbol', 'date', 'metric', 'value']]
df_long = df_long.sort_values(['symbol', 'date', 'metric'])

print(f"Ready: {len(df_long):,} rows")

# ====================== 2. CONNECT ======================
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Ensure DB exists
try:
    with engine.connect(): pass
except OperationalError:
    tmp = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres")
    with tmp.connect() as conn:
        conn.execute("COMMIT")
        conn.execute(text(f"CREATE DATABASE {DB_NAME}"))

# ====================== 3. CREATE TABLE ======================
with engine.begin() as conn:
    conn.execute(text(f"""
        DROP TABLE IF EXISTS {TABLE_NAME};
        CREATE TABLE {TABLE_NAME} (
            symbol  VARCHAR(10)   NOT NULL,
            date    DATE         NOT NULL,
            metric  VARCHAR(6)    NOT NULL CHECK (metric IN ('open','high','low','close','volume')),
            value   DOUBLE PRECISION,
            PRIMARY KEY (symbol, date, metric)
        );
        CREATE INDEX idx_symbol ON {TABLE_NAME}(symbol);
        CREATE INDEX idx_date   ON {TABLE_NAME}(date);
    """))
print(f"Table `{TABLE_NAME}` created.")

# ====================== 4. LOAD (FAST + REPLACE) ======================
print("Loading data...")
df_long.to_sql(
    TABLE_NAME,
    engine,
    if_exists='append',   # or 'replace' if you want to wipe first
    index=False,
    method='multi',
    chunksize=10_000
)
print(f"Done! Loaded {len(df_long):,} rows.")

Loading and melting CSV...
Ready: 1,262,520 rows
Table `sp500_historical_data` created.
Loading data...
Done! Loaded 1,262,520 rows.
